In [1]:
import numpy as np 
import pandas as pd 
from sklearn.metrics.pairwise import linear_kernel
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re
import string

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
trainning_df = pd.read_excel('/kaggle/input/gov-tn/Classeur1.xltx')

In [3]:
trainning_df.head()

,website,paragraph,question
0,http://www.finances.gov.tn/fr/,Le décret n° 75-316 du 30 mai 1975 fixe les at...,C'est quoi le ministère de la finance et quell...
1,NaN,Création :\nLe CNF est crée par l’article 4 du...,C'est quoi le sonseil national de la fiscalité...
2,NaN,Mme Sihem Boughdiri Nemsia nommée par le prési...,Qui est la minitstre des finances
3,NaN,Le conseil supérieur de comptabilité a été cré...,C'est quoi le conseil de comptabilité ?
4,NaN,Le Conseil National de la Comptabilité est com...,Quelle est la composition du conseil de la com...


In [4]:
trainning_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   website    5 non-null      object
 1   paragraph  227 non-null    object
 2   question   227 non-null    object
dtypes: object(3)
memory usage: 5.4+ KB


In [5]:
trainning_df.isnull().sum()

website      222
paragraph      0
question       0
dtype: int64

In [6]:
trainning_question_ls = list(trainning_df['question'])
trainning_answer_ls = list(trainning_df['paragraph'])


In [7]:
print(len(trainning_answer_ls))
print(len(trainning_question_ls))

227
227


answerdata augmentation 

In [8]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(trainning_question_ls).reshape(-1, 1), np.array(trainning_answer_ls).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['question', 'answer']);

In [9]:
#train_os.info()

In [10]:
#train_os.head()

In [11]:
# Eliminer les stop words: les mots très fréquents en anglais
nltk.download('stopwords')
final_stopwords_list = stopwords.words('english') + stopwords.words('french')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if not (word.lower() in final_stopwords_list)])

In [13]:
#Eliminer les ponctuations
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [14]:
#Eliminer les nombres
def cleaning_numbers(data):
    return re.sub('[0-9]+','', data)

In [15]:
def strip_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\t', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    banned_list= string.punctuation +'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

In [16]:
keywords=list()
for item in train_os['answer'] :
    item_num=cleaning_numbers(item)
    item_semi_clean=cleaning_punctuations(item_num)
    item_semi=strip_entities(item_semi_clean)
    item_clean=cleaning_stopwords(item_semi)
    # Initialize the TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words=final_stopwords_list)

    # Fit the TfidfVectorizer to the text
    tfidf_vectorizer.fit([item_clean])

    # Get the feature names and their corresponding scores
    feature_names = tfidf_vectorizer.get_feature_names_out() 
    feature_scores = tfidf_vectorizer.idf_

    # Combine the feature names and their corresponding scores into a dictionary
    feature_dict = dict(zip(feature_names, feature_scores))

    # Sort the dictionary by the feature scores in descending order
    sorted_features = sorted(feature_dict.items(), key=lambda x: x[1], reverse=True)
    keywords.append(sorted_features[0][0]+','+sorted_features[1][0]+','+sorted_features[2][0])

In [17]:
print(keywords)

['accorde,accords,administratif', 'agriculture,article,artisanat', 'administration,afférentes,ailleurs', 'article,attribué,composition', 'administratif,banque,bvmt', 'auprès,auxiliaires,besoin', 'amélioration,application,avis', 'accordé,accounting,adoptée', 'accorder,achat,activité', 'adéquates,ainsi,appréciation', 'approbation,arrêtés,cadre', 'affaires,affectés,archives', 'affaires,agricole,ainsi', 'agence,agricole,alcools', 'accès,administratif,ar', 'acquis,administration,admis', 'ainsi,aligner,amélioration', 'afférentes,ainsi,application', 'administration,administrative,adopte', 'ajouté,août,approbation', 'ar,cette,commission', 'avis,cnncp,collectivités', 'accès,action,adapter', 'avis,cnncp,comptes', 'année,besoins,budget', 'ajoutée,assurances,certains', 'abattements,absence,accordés', 'acomptes,actions,actionsau', 'acceptés,accompagnement,achats', 'ad,alcool,alcoolisées', 'accompagnée,accordés,acheter', 'agricoles,assurance,autres', 'activité,affaires,aménagement', 'ajoutée,août,av

In [21]:
df = pd.DataFrame(columns=['question','answer','Keywords'])
df['question'] =['question: '+item for item in train_os['question']]
df['answer'] = ['answer: '+item for item in train_os['answer']]
df['Keywords']=[' Keywords : '+ keyword for keyword in keywords]

In [22]:
df_final = pd.DataFrame(columns=['question','answer'])
df_final['question'] = trainning_question_ls
df_final['answer']=df['answer']+ df['Keywords']

In [23]:
df_final['answer'][0]

'answer: Le décret n° 75-316 du 30 mai 1975 fixe les attributions du Ministère des Finances qui a pour mission essentielle l’élaboration et la mise en œuvre de la politique de l’Etat en matière financière, monétaire et fiscale. Dans ce cadre, il participe à la conception des objectifs de la politique économique et à la définition des moyens de réalisation appropriés. \n\nLe Ministère des Finances a aussi pour mission :\n\nD’élaborer les projets des lois de finances et de veiller, conformément à la législation et à la réglementation en vigueur à l’exécution du budget de l’Etat, des budgets annexes, des budgets des établissements publics à caractère administratif, ainsi que des fonds spéciaux du trésor. A ce titre, il est consulté sur toutes les questions ayant une incidence budgétaire et notamment celles afférentes aux rémunérations publiques ;\nDe préparer les projets de textes à caractère fiscal et douanier et assure l’exécution de la législation dans ce domaine ;\nDe préparer les pro

In [24]:
df.head()

,question,answer,Keywords
0,question: C'est quoi le ministère de la financ...,answer: Le décret n° 75-316 du 30 mai 1975 fix...,"Keywords : accorde,accords,administratif"
1,question: C'est quoi le sonseil national de la...,answer: Création :\nLe CNF est crée par l’arti...,"Keywords : agriculture,article,artisanat"
2,question: Qui est la minitstre des finances,answer: Mme Sihem Boughdiri Nemsia nommée par ...,"Keywords : administration,afférentes,ailleurs"
3,question: C'est quoi le conseil de comptabilité ?,answer: Le conseil supérieur de comptabilité a...,"Keywords : article,attribué,composition"
4,question: Quelle est la composition du conseil...,answer: Le Conseil National de la Comptabilité...,"Keywords : administratif,banque,bvmt"


In [25]:
df_final.head()

,question,answer
0,C'est quoi le ministère de la finance et quell...,answer: Le décret n° 75-316 du 30 mai 1975 fix...
1,C'est quoi le sonseil national de la fiscalité...,answer: Création :\nLe CNF est crée par l’arti...
2,Qui est la minitstre des finances,answer: Mme Sihem Boughdiri Nemsia nommée par ...
3,C'est quoi le conseil de comptabilité ?,answer: Le conseil supérieur de comptabilité a...
4,Quelle est la composition du conseil de la com...,answer: Le Conseil National de la Comptabilité...


**MODEL**

In [26]:
!pip install transformers==4.14.1 -q
!pip install bitsandbytes-cuda111==0.26.0 -q
!pip install datasets==1.16.1 -q

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers
      creating build/lib.linux-x86_64-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/models
      creating build/lib.linux-x86_64-cpython-310/tokenizers/decoders
      copying py_src/tokenizers/decoders/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/decoders
      creating build/lib.linux-x86_64-cpython-310/tokenizers/normalizers
      copying py_src/tokenizers/normalizers/__init__.py -> build/lib.linux-x86_64-cpython-310/tokenizers/normalizer

In [27]:
from sklearn.model_selection import train_test_split
import transformers
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd
from tqdm.auto import tqdm

In [28]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias).clone()
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr(
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [29]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock

In [30]:
class T5ForConditionalGeneration(transformers.models.t5.modeling_t5.T5ForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration = T5ForConditionalGeneration

In [31]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

In [32]:
config.pad_token_id = config.eos_token_id
tokenizer.pad_token = config.pad_token_id

In [33]:
#gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)
gpt = GPTJForCausalLM.from_pretrained("gustavecortal/fr-boris-8bit", low_cpu_mem_usage=True) 

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

In [34]:
!gdown --id 1Q1WMjny26VHLKb71iTCHIS5zvdm9c-wz

/bin/bash: gdown: command not found


In [35]:
data=pd.DataFrame()
data['sentence'] = '[Sentence]:'+'en Tunisie,'+train_os['question']+'\n[Paraphrase]:'+'en Tunisie,'+train_os['answer']
data=data['sentence']
train, test = train_test_split(data, test_size=0.05) 
train.to_csv('/kaggle/working/train_pgbp_example.csv', index=False)
test.to_csv('/kaggle/working/test_pgbp_example.csv', index=False)
data.head()

0    [Sentence]:en Tunisie,C'est quoi le ministère ...
1    [Sentence]:en Tunisie,C'est quoi le sonseil na...
2    [Sentence]:en Tunisie,Qui est la minitstre des...
3    [Sentence]:en Tunisie,C'est quoi le conseil de...
4    [Sentence]:en Tunisie,Quelle est la compositio...
Name: sentence, dtype: object

In [36]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': '/kaggle/working/train_pgbp_example.csv',
                                              'test': '/kaggle/working/test_pgbp_example.csv'})

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-140a69c2a44ef934/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:170: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:170: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

In [37]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding=True, truncation=True, max_length= 128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["sentence"])
tokenized_datasets.set_format("torch")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
from torch.utils.data import DataLoader

full_train_dataset = tokenized_datasets["train"]
train_dataloader = DataLoader(full_train_dataset, shuffle=False, batch_size=8)

In [39]:
def add_adapters(model, adapter_dim=4, p = 0.1):
    assert adapter_dim > 0

    for name, module in model.named_modules():
      if isinstance(module, FrozenBNBLinear):
          if "attn" in name or "mlp" in name or "head" in name:
              print("Adding adapter to", name)
              module.adapter = nn.Sequential(
                nn.Linear(module.in_features, adapter_dim, bias=False),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.out_features, bias=False),
            )
              print("Initializing", name)
              nn.init.zeros_(module.adapter[2].weight)

          else:
              print("Not adding adapter to", name)
      elif isinstance(module, FrozenBNBEmbedding):
          print("Adding adapter to", name)
          module.adapter = nn.Sequential(
                nn.Embedding(module.num_embeddings, adapter_dim),
                nn.Dropout(p=p),
                nn.Linear(adapter_dim, module.embedding_dim, bias=False),
            )
          print("Initializing", name)
          nn.init.zeros_(module.adapter[2].weight)

add_adapters(gpt)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
gpt.to(device)

Adding adapter to transformer.wte
Initializing transformer.wte
Adding adapter to transformer.h.0.attn.k_proj
Initializing transformer.h.0.attn.k_proj
Adding adapter to transformer.h.0.attn.v_proj
Initializing transformer.h.0.attn.v_proj
Adding adapter to transformer.h.0.attn.q_proj
Initializing transformer.h.0.attn.q_proj
Adding adapter to transformer.h.0.attn.out_proj
Initializing transformer.h.0.attn.out_proj
Adding adapter to transformer.h.0.mlp.fc_in
Initializing transformer.h.0.mlp.fc_in
Adding adapter to transformer.h.0.mlp.fc_out
Initializing transformer.h.0.mlp.fc_out
Adding adapter to transformer.h.1.attn.k_proj
Initializing transformer.h.1.attn.k_proj
Adding adapter to transformer.h.1.attn.v_proj
Initializing transformer.h.1.attn.v_proj
Adding adapter to transformer.h.1.attn.q_proj
Initializing transformer.h.1.attn.q_proj
Adding adapter to transformer.h.1.attn.out_proj
Initializing transformer.h.1.attn.out_proj
Adding adapter to transformer.h.1.mlp.fc_in
Initializing transfor

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): FrozenBNBLinear(4096, 50400)
)

In [40]:
!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 11.0 MB/s eta 0:00:0000:0100:01


In [41]:
from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu')}
  warn(msg)


In [42]:
from bitsandbytes.optim import Adam8bit

gpt.gradient_checkpointing_enable()
optimizer = Adam8bit(gpt.parameters(), lr=1e-5, weight_decay=0.01)

In [43]:
num_epochs = 5
num_training_steps = num_epochs * len(train_dataloader)

In [44]:
lr_scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer, int(num_training_steps*0.1), num_training_steps
)

In [45]:
filepath = '/kaggle/working/model.pt'

In [46]:
from tqdm.auto import tqdm

scaler = torch.cuda.amp.GradScaler()
progress_bar = tqdm(range(num_training_steps))
gpt.train()
gpt.gradient_checkpointing_enable()
k = 0

for epoch in range(num_epochs):
    for batch in train_dataloader:

        k = k + 1
        if k % 500 == 0:
          print(k)
          state = {'k' : k, 'epoch': num_epochs, 'lr_scheduler': lr_scheduler.state_dict(), 'state_dict': gpt.state_dict(), 'optimizer': optimizer.state_dict()}
          torch.save(state, filepath)

        batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
          out = gpt.forward(**batch,)

          loss = F.cross_entropy(out.logits[:, :-1, :].flatten(0, -2), batch['input_ids'][:, 1:].flatten(),
                                reduction='mean', label_smoothing=0.1)
          
        print(loss)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(gpt.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        lr_scheduler.step()
        progress_bar.update(1)

  0%|          | 0/270 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


tensor(3.9596, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3571, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.7591, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9664, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.3688, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9823, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8529, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.6525, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.1560, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9713, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4302, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9368, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9443, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.1735, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.2613, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.7154, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.0299, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.9205, device='cuda:0', grad_fn=<AddBack

set the question !!!! 

In [47]:
# Compute TF-IDF matrix
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(train_os['answer'])

In [48]:
gpt.eval()
with torch.no_grad():
  prompt = tokenizer("question : en tunisie quels sont les impôts ? , Exemple of answer : " , truncation=True, padding=True, max_length=128, return_tensors='pt')
  prompt = {key: value.to(device) for key, value in prompt.items()}
  out = gpt.generate(**prompt, max_length=128, top_k=50, top_p=0.9, temperature=1.0, do_sample=True, repetition_penalty = 1.2, num_beams=1)
  user_input=tokenizer.decode(out[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [49]:
# Transform user input into TF-IDF features
user_tfidf = tfidf.transform([user_input])

# Compute cosine similarity between user input and listings
cosine_similarities = linear_kernel(user_tfidf, tfidf_matrix).flatten()

# Get indices of top 5 listings with highest similarity scores
top_indices = cosine_similarities.argsort()[:-6:-1]

# Recommend the most suitable listings
recommended_listings = df.loc[top_indices]

In [50]:
# Display the recommended listings
for index, listing in recommended_listings.iterrows():
    print("Choix: {}".format(index))
    print("Answer: {}".format(listing['answer']))
    print("-----------------------")

Choix: 49
Answer: answer: Le cadre légal et règlementaire dans lequel opère la comptabilité publique est très vaste compte tenu de la multitude et de la variété des opérations financières et comptables induites par l'activité des entités publiques. Les textes les plus importants sont :

Textes législatifs :

Code de la comptabilité publique promulgué par la loi n°73-81 du 31 décembre 1973, tel que modifié notamment par la loi des finances pour la gestion 2010;
Loi n°67-53 du 8 décembre 1967 portant loi organique du budget telle que modifiée notamment par la loi n°2004-42 du 13 mai 2004;
Loi 75-35 du 14 mai 1975 portant loi organique du budget des collectivités locales telle que modifiée notamment par la loi organique n°2007-65 du 18 décembre 2007;
Loi 68-8 du 8 mars 1968 portant organisation de la cour des comptes telle que modifiée notamment par la loi organique n°2008-3 du 29 janvier 2008;
Code de la TVA promulgué par la loi N°88-61 du 2 Juin 1988;
Loi N°88-62 du 2 Juin 1988 Portant 

In [51]:
test.values[0].split('[Paraphrase]:')[0].strip()

'[Sentence]:en Tunisie,Quelle est la période vérifiable?'

In [52]:
gpt.eval()
for sentence in test.values:
  st = sentence.split('[Paraphrase]:')[0].strip()
  with torch.no_grad():
    prompt = tokenizer(st, truncation=True, padding=True, max_length=128, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = gpt.generate(**prompt, max_length=512, top_k=50, top_p=0.9, temperature=1.0, do_sample=True, repetition_penalty = 1.2, num_beams=1)
    print('\n')
    print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,Quelle est la période vérifiable?
[Paraphrase]:en Tunisie,La période de surveillance fiscale couvre la période allant du 1er octobre au 30 septembre de l'année suivante.<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,que sont les points important sur la conférences de sensibilisation sur l’Accord de la Zone de Libre-échange continentale africaine (ZLECAf)?
[Paraphrase]:en Tunisie,La Conférence des Ministres Africains en charge du Commerce s'est tenue le 10 Juillet 2019 à Kinshasa pour finaliser la liste des produits concernés par la mise en œuvre du code de transparence et pour apporter les précisions sur les éventuels barrièrestructurants pour le développement de la filière industrielle en Afrique. La commission a discuté notamment du déroulement de la négociation de la ZLESCAf au cours de laquelle la ZLESCAf a été définie, la feuille de route sur la mise en œuvre des dispositions relatives à la protection du consommateur, la question de l'accélération du processus législatif de l'accord. La Commission a aussi apporté une révision des amendements adoptés durant la négociation et les études des questions relatives aux énergies renouvelables. Par ailleurs, la Commission des Q

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...



[Sentence]:en Tunisie,Quelles sont les obligations des assujettis à la TVA relatives à la facturation?
[Paraphrase]:en Tunisie,Les personnes physiques et morales sont astreintes à la facturation en vue de la perception de la TVA.Elles ne peuvent se dispenser de la facturer.La facture doit être délivrée et conservée par le prestataire du service.<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,Quels sont les droits d’enregistrement exigibles pour l’acquisition d’habitation auprès d’un promoteur immobilier?
[Paraphrase]:en Tunisie,Les droits d'enregistrements s'élèvent à 10% du prix du logement à la charge de l'acquéreur.<|endoftext|>


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




[Sentence]:en Tunisie,Quels sont les agents habilités à procéder à la vérification fiscale?
[Paraphrase]:en Tunisie,Les agents de l’administration fiscale en charge du contrôle de la comptabilité des personnesmorales sont habiles à faire toutes vérifications dans les entreprises sur place ou à distance ainsi qu’en leurs locaux. Toutefois ils ne peuvent recourir à cette possibilité qu’avec l'autorisation du chef d'agence ou de la direction régionnal.<|endoftext|>


[Sentence]:en Tunisie,Quelles sont les obligations déclaratives à la fin de l'activité?
[Paraphrase]:en Tunisie,La cessation définitive d'exploitation a pour effet de provoquer un état de liquidation, sans interruption de la comptabilité. Les entreprises, dont les droits découlent d'activités poursuivies, doivent en tenir état par le dépôt d'une déclaration accompagnée des pièces justificatives au registre de commerce et des sociétés ou auprès du services des impôts chargé de ladite assimilation. Les autres personnes physiq